Part 1 - Data Cleaning

In [ ]:
#import libraries
import pandas as pd

df = pd.read_csv("../data/online_shoppers_intention_data.csv")
df.columns = df.columns.str.strip()  


In [7]:
#splitting data 
#splitting dataset into user sessions
user_sessions = df[['Administrative', 'Informational', 'ProductRelated',
                    'BounceRates', 'ExitRates', 'PageValues',
                    'VisitorType', 'Weekend', 'OperatingSystems', 'Browser',
                    'Region', 'TrafficType', 'Month']].copy()
user_sessions['session_id'] = user_sessions.index


In [13]:
#splitting datset into traffic source
traffic_source = df[['TrafficType', 'VisitorType', 'Weekend', 'Region', 'Month']].copy()
traffic_source['session_id'] = traffic_source.index
traffic_source = traffic_source.rename(columns={
    'TrafficType': 'campaign_id',
    'VisitorType': 'channel',
    'Region': 'geo_region'
})


In [15]:
#splitting dataset into transactions
transactions = df[['PageValues', 'Revenue']].copy()
transactions['session_id'] = transactions.index
transactions = transactions.rename(columns={
    'Revenue': 'purchase_flag',
    'PageValues': 'cart_value'
})


In [17]:
#Joining traffic source and transactions by session_Id
merged_df = user_sessions.merge(traffic_source, on='session_id') \
                         .merge(transactions, on='session_id')


In [19]:
#Add columns to Data 
# Define each funnel stage as a True/False flag (Boolean)
merged_df['product_view'] = merged_df['ProductRelated'] > 0
merged_df['add_to_cart'] = merged_df['PageValues'] > 0
merged_df['purchase'] = merged_df['purchase_flag'] == True




In [21]:
#Export 3 tables and funnel data to excel
user_sessions.to_excel("user_sessions.xlsx", index=False)
traffic_source.to_excel("traffic_source.xlsx", index=False)
transactions.to_excel("transactions.xlsx", index=False)
merged_df.to_excel("funnel_table.xlsx", index=False)


Part 2 - SQL Queries


In [27]:
import sqlite3
import pandas as pd

# Loading funnel Excel
df = pd.read_excel("../Excel/funnel_table.xlsx")

# Creating SQLite and upload
conn = sqlite3.connect(":memory:")
df.to_sql("funnel_data", conn, index=False, if_exists="replace")


12330

In [35]:
# Funnel Analysis with SQL query
basic_funnel_query = """
SELECT
  COUNT(*) AS total_sessions,
  SUM(CASE WHEN product_view = 1 THEN 1 ELSE 0 END) AS product_views,
  SUM(CASE WHEN add_to_cart = 1 THEN 1 ELSE 0 END) AS add_to_cart,
  SUM(CASE WHEN purchase = 1 THEN 1 ELSE 0 END) AS purchases
FROM funnel_data;
"""

funnel_basic = pd.read_sql(basic_funnel_query, conn)
display(funnel_basic)



,total_sessions,product_views,add_to_cart,purchases
0,12330,12292,2730,1908


In [37]:
# Calculate Conversion Rate
conversion_query = """
SELECT
  ROUND(SUM(product_view) * 1.0 / COUNT(*) * 100, 2) AS product_view_rate,
  ROUND(SUM(add_to_cart) * 1.0 / SUM(product_view) * 100, 2) AS cart_conversion_rate,
  ROUND(SUM(purchase) * 1.0 / SUM(add_to_cart) * 100, 2) AS purchase_conversion_rate
FROM funnel_data
WHERE product_view = 1;  -- Ensures denominators are safe
"""

conversion_rates = pd.read_sql(conversion_query, conn)
display(conversion_rates)



,product_view_rate,cart_conversion_rate,purchase_conversion_rate
0,100.0,22.21,69.67


In [39]:
#Grouped Funnel Performance by Campaign or Channel
grouped_funnel_query = """
SELECT
  campaign_id,
  channel,
  COUNT(*) AS total_sessions,
  SUM(product_view) AS product_views,
  SUM(add_to_cart) AS add_to_cart,
  SUM(purchase) AS purchases,
  ROUND(SUM(product_view) * 1.0 / COUNT(*) * 100, 2) AS view_rate,
  ROUND(SUM(add_to_cart) * 1.0 / SUM(product_view) * 100, 2) AS cart_rate,
  ROUND(SUM(purchase) * 1.0 / SUM(add_to_cart) * 100, 2) AS purchase_rate
FROM funnel_data
GROUP BY campaign_id, channel
ORDER BY purchases DESC
LIMIT 10;
"""

funnel_by_campaign = pd.read_sql(grouped_funnel_query, conn)
display(funnel_by_campaign)


,campaign_id,channel,total_sessions,product_views,add_to_cart,purchases,view_rate,cart_rate,purchase_rate
0,2,Returning_Visitor,2966,2958,889,608,99.73,30.05,68.39
1,1,Returning_Visitor,2407,2407,463,255,100.00,19.24,55.08
2,2,New_Visitor,936,936,216,238,100.00,23.08,110.19
3,3,Returning_Visitor,1905,1898,251,154,99.63,13.22,61.35
4,4,Returning_Visitor,970,969,250,132,99.90,25.80,52.80
5,10,Returning_Visitor,425,424,102,85,99.76,24.06,83.33
6,8,New_Visitor,175,173,41,50,98.86,23.70,121.95
7,8,Returning_Visitor,167,165,42,45,98.80,25.45,107.14
8,13,Returning_Visitor,731,731,105,43,100.00,14.36,40.95
9,6,Returning_Visitor,401,397,91,42,99.00,22.92,46.15


In [41]:
# Exporting summary to Excel
funnel_by_campaign.to_excel("funnel_campaign_summary.xlsx", index=False)


In [51]:
#Cleaning data for Power BI
df = pd.read_excel("../Excel/funnel_table.xlsx")

#dropping unneccessary columns
df.drop(columns=['Weekend_x', 'Month_x', 'VisitorType', 'purchase_flag'], inplace=True)

#renaming column names for clarity
df.rename(columns={
    'Month_y': 'Month',
    'Weekend_y': 'Weekend'
}, inplace=True)

#Converting funnel flags to 0/1
funnel_flags = ['product_view', 'add_to_cart', 'purchase']

for col in funnel_flags:
    df[col] = df[col].astype(int)


In [53]:
#Save cleaned data for Power BI
df.to_excel("funnel_cleaned_for_pbi.xlsx", index=False)


In [54]:
df.info()
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Administrative    12330 non-null  int64  
 1   Informational     12330 non-null  int64  
 2   ProductRelated    12330 non-null  int64  
 3   BounceRates       12330 non-null  float64
 4   ExitRates         12330 non-null  float64
 5   PageValues        12330 non-null  float64
 6   OperatingSystems  12330 non-null  int64  
 7   Browser           12330 non-null  int64  
 8   Region            12330 non-null  int64  
 9   TrafficType       12330 non-null  int64  
 10  session_id        12330 non-null  int64  
 11  campaign_id       12330 non-null  int64  
 12  channel           12330 non-null  object 
 13  Weekend           12330 non-null  bool   
 14  geo_region        12330 non-null  int64  
 15  Month             12330 non-null  object 
 16  cart_value        12330 non-null  float6

,Administrative,Informational,ProductRelated,BounceRates,ExitRates,PageValues,OperatingSystems,Browser,Region,TrafficType,session_id,campaign_id,channel,Weekend,geo_region,Month,cart_value,product_view,add_to_cart,purchase
0,0,0,1,0.20,0.20,0.0,1,1,1,1,0,1,Returning_Visitor,False,1,Feb,0.0,1,0,0
1,0,0,2,0.00,0.10,0.0,2,2,1,2,1,2,Returning_Visitor,False,1,Feb,0.0,1,0,0
2,0,0,1,0.20,0.20,0.0,4,1,9,3,2,3,Returning_Visitor,False,9,Feb,0.0,1,0,0
3,0,0,2,0.05,0.14,0.0,3,2,2,4,3,4,Returning_Visitor,False,2,Feb,0.0,1,0,0
4,0,0,10,0.02,0.05,0.0,3,3,1,4,4,4,Returning_Visitor,True,1,Feb,0.0,1,0,0
